In [1]:
import os
import torch
import pandas as pd
import spacy
from spacy.tokens import SpanGroup
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

In [2]:
SCRIPT_DIR = os.getcwd()
DATA_PATH = os.path.join(SCRIPT_DIR, "PET_dataset.csv")

data = pd.read_csv(DATA_PATH)
data["tokens"] = data["tokens"].apply(eval)
data["ner_tags"] = data["ner_tags"].apply(eval)

In [3]:
nlp_default = spacy.load("en_core_web_sm")

try:
    nlp_coref = spacy.load("en_core_web_trf")
    nlp_coref.add_pipe("coreferee")
except Exception as e:
    nlp_coref = spacy.load("en_core_web_sm")

In [4]:
IGNORE_WORDS = {"my", "mine", "his", "her", "hers", "its", "their", "theirs", "ours", "our", "your", "yours", "this", "that", "these", "those"}

def process_text(text):
    doc1 = nlp_default(text)
    doc2 = nlp_coref(text)
    return doc1, doc2

def resolve_references(text: str) -> str:
    doc1, doc2 = process_text(text)
    if not doc2.spans:
        return text
    token_mention_mapper = {}
    clusters = [val for key, val in doc2.spans.items() if key.startswith("coref_cluster")]
    for cluster in clusters.copy():
        for span in cluster:
            if doc1[span.start].pos_ == "VERB":
                clusters.remove(cluster)
                break
    new_clusters = []
    for i, cluster in enumerate(clusters):
        new_cluster = SpanGroup(doc2, name=f"coref_cluster_{i+1}")
        for span in cluster:
            new_cluster.append(span)
        new_clusters.append(new_cluster)
    for cluster in new_clusters:
        first_mention = cluster[0]
        for mention_span in list(cluster)[1:]:
            token_mention_mapper[mention_span[0].idx] = first_mention.text + mention_span[0].whitespace_
            for token in mention_span[1:]:
                token_mention_mapper[token.idx] = ""
    output_string = ""
    for token in doc2:
        output_string += token_mention_mapper.get(token.idx, token.text + token.whitespace_)
    return output_string

In [5]:
model_name = "jtlicardo/bpmn-information-extraction"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
device = 0 if torch.cuda.is_available() else -1
bert_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=device)
print(f"Using device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

Using device: GPU


In [6]:
all_texts = []
for tokens in data["tokens"]:
    raw_text = " ".join(tokens)
    resolved_text = resolve_references(raw_text)
    all_texts.append(resolved_text)
data["all_extracted_entities"] = [bert_pipeline(text) for text in all_texts]

C:\Users\hello\Anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [7]:
nlp_sm = spacy.load("en_core_web_sm")
def refine_entities(entities):
    if not entities:
        return None
    refined = []
    for ent in entities:
        doc = nlp_sm(ent["word"])
        if len(doc) >= 2:
            refined.append(ent["word"])
    return refined if refined else None
data["refined_entities"] = data["all_extracted_entities"].apply(refine_entities)

In [8]:
def extract_gold_tasks(tokens, ner_tags):
    gold_tasks = []
    current = []
    for token, tag in zip(tokens, ner_tags):
        if tag.startswith("B-Activity"):
            if current:
                gold_tasks.append(" ".join(current))
            current = [token]
        elif tag.startswith("I-Activity") and current:
            current.append(token)
        else:
            if current:
                gold_tasks.append(" ".join(current))
                current = []
    if current:
        gold_tasks.append(" ".join(current))
    return gold_tasks if gold_tasks else None
data["gold_tasks"] = data.apply(lambda row: extract_gold_tasks(row["tokens"], row["ner_tags"]), axis=1)

In [9]:
def normalize_text(text):
    doc = nlp_sm(text.lower().strip())
    return " ".join(token.lemma_ for token in doc)

def normalize_list(lst):
    return set(normalize_text(item) for item in lst) if lst else set()

def evaluate_fuzzy(predicted, gold):
    p_list = list(normalize_list(predicted))
    g_list = list(normalize_list(gold))
    matches = 0
    used_pred_indices = set()
    for gold_task in g_list:
        for idx, pred_task in enumerate(p_list):
            if idx in used_pred_indices:
                continue
            if gold_task in pred_task or pred_task in gold_task:
                matches += 1
                used_pred_indices.add(idx)
                break
    tp = matches
    fp = len(p_list) - matches
    fn = len(g_list) - matches
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

In [10]:
data["precision"], data["recall"], data["f1_score"] = zip(*data.apply(lambda row: evaluate_fuzzy(row["refined_entities"], row["gold_tasks"]), axis=1))
overall_precision = data["precision"].mean()
overall_recall = data["recall"].mean()
overall_f1 = data["f1_score"].mean()

print("\nOverall Precision: {:.4f}".format(overall_precision))
print("Overall Recall:    {:.4f}".format(overall_recall))
print("Overall F1-score:  {:.4f}".format(overall_f1))


Overall Precision: 0.5092
Overall Recall:    0.6765
Overall F1-score:  0.5740
